# Neural Traces Data Preprocessing Pipeline

Goals:

* HMM & RNN - compatible data format (# neurons, traces in timeframes)

In [1]:
import pickle
import numpy as np

In [2]:
with open('./data/leah_20200206_07_08_09_10_11_12_13.pickle', 'rb') as pickle_file:
    data = pickle.load(pickle_file)

The data has the following structure:

- all_data:
  - was_completed
  - was_correct
  - stim_dir
  - correct_side
  - response_side
  - prior
  - noise
  - traces
  - task_info
  - frame_info

- completed_trials_data:
  - was_completed
  - was_correct
  - stim_dir
  - correct_side
  - prior
  - noise
  - response_side
  - traces_stim_aligned
  - traces_resp_aligned
  - completed_inds
  - frame_info
  
It doesn't make sense to look at all_data, let's just do `completed_trials`.

In [3]:
for i in range(len(data)):
    data[i]['completed_trials_data']['traces'] = data[i]['all_data']['traces']
    data[i] = data[i]['completed_trials_data']
    del data[i]['was_completed'] # we know it's completed (I also checked before)
    del data[i]['correct_side'] # same as stim_dir
    del data[i]['traces_resp_aligned']
    del data[i]['traces_stim_aligned']

### Create the inputs

In [4]:
# creates the u_in object (8, # timeframes)
def process_stim_trials(stim_trials, u_in, start_frames, end_frames, stim_dict, no_stims):
    issues_detected = []

    for i, stimulus in enumerate(stim_trials):
        stim_vector = np.zeros(no_stims)
        stim_vector[stim_dict[stimulus]] = 1
        
        start_frame = int(start_frames[i])
        end_frame = int(end_frames[i])

        # Check for mismatched start and end frames
        if start_frame == end_frame:
            issues_detected.append(f"Issue at index {i}: Start frame is the same as end frame ({start_frame}).")
            continue  # Skip this iteration

        # Ensure correct ordering of frames
        if start_frame > end_frame:
            issues_detected.append(f"Issue at index {i}: Start frame ({start_frame}) is greater than end frame ({end_frame}).")
            continue  # Skip this iteration

        stim_length = end_frame - start_frame

        # Attempt to assign the values
        try:
            u_in[:, start_frame:end_frame] = np.tile(stim_vector[:, np.newaxis], (1, stim_length))
        except ValueError as ve:
            issues_detected.append(f"Issue at index {i}: {ve}")

    return issues_detected, u_in

In [5]:
stim_dict = { # this will be used for the one-hot encoding
    -1.0: 0, 
    -0.75: 1, 
    -0.5: 2, 
    -0.25: 3, 
    0.25: 4, 
    0.5: 5, 
    0.75: 6, 
    1.0: 7}

no_stims = 8

for i in range(len(data)):
    # transform left and right into -1 and 1
    stim_dir = data[i]['stim_dir']
    stim_dir = np.array([-1 if drn == 'left' else 1 for drn in stim_dir])
    stim_trial = np.empty(data[i]['noise'].shape)
    stim_trial = stim_dir * data[i]['noise'] # this captures both the direction and the noise level

    len_neural_traces = data[i]['traces'].shape[1]
    u_in = np.zeros((no_stims, len_neural_traces))
    print(f'Input shape: {u_in.shape}')
    
    start_frames = data[i]['frame_info']['stim_start_frame']
    end_frames = data[i]['frame_info']['stim_end_frame']
    issues, u_in = process_stim_trials(stim_trial, u_in, start_frames, end_frames, stim_dict, no_stims)
    data[i]['u_in'] = u_in

Input shape: (8, 35745)
Input shape: (8, 20905)
Input shape: (8, 37961)
Input shape: (8, 37717)
Input shape: (8, 37717)
Input shape: (8, 32876)
Input shape: (8, 35225)


### Just trials

Take the `stim_start_frame` and the `response_frame` (+20 timeframes for the reward and end of trial), adjust `traces` and `u_in`.

In [6]:
for idx in range(len(data)):

    trial_traces = None
    trial_inputs = None
    s = data[idx]['frame_info']['stim_start_frame']
    e = data[idx]['frame_info']['response_frame'] + 20

    for start, end in zip(s, e):
        if (trial_traces is None) and (trial_inputs is None):
            trial_traces = data[idx]['traces'][:, int(start): int(end)]
            trial_inputs = data[idx]['u_in'][:, int(start): int(end)]
        else:
            t = data[idx]['traces'][:, int(start): int(end)]
            i = data[idx]['u_in'][:, int(start): int(end)]
            trial_traces = np.concatenate((trial_traces, t), axis=1)
            trial_inputs = np.concatenate((trial_inputs, i), axis=1)

    data[idx]['trial_traces'] = trial_traces
    data[idx]['trial_u_in'] = trial_inputs

## Save the data

In [7]:
with open('./data/preprocessed_data.pickle', 'wb') as f:
    pickle.dump(data, f)

## Optional adjustments

Further preprocessing: 

* Gaussian smoothing
* normalization with max and mean

Note: this will be done just before training

In [8]:
import numpy as np
from scipy.ndimage import gaussian_filter1d
from skimage.measure import block_reduce

In [9]:
SIGMA = 10 # needs to be experimented with

# for CORNN, you need two sets of traces, off-set by 1 timeframe (inputs and targets)
traces_raw = data[0]['traces']
traces_raw = traces_raw / np.max(traces_raw,axis = 1)[:,None]
traces_raw = traces_raw - np.mean(traces_raw,axis = 1)[:,None]

traces_raw2 = data[0]['traces']
traces_raw2 = traces_raw2 / np.max(traces_raw2,axis = 1)[:,None]
traces_raw2 = traces_raw2 - np.mean(traces_raw2,axis = 1)[:,None]

r_in = gaussian_filter1d(traces_raw, sigma=SIGMA) # just so that CORNN targets are offset by 1
r_tar = gaussian_filter1d(traces_raw2, sigma=SIGMA)

u_in = gaussian_filter1d(data[0]['u_in'], sigma=SIGMA)

In [ ]:
def normalize_traces(traces):
    """
    Normalize neural traces values to lie between -1 and 1.
    """
    t = traces / np.max(traces, axis = 1)[:,None]
    t = t - np.mean(t, axis = 1)[:,None]
    return t

def full_data_prep(traces_raw, traces_raw_offset, u_in_raw, SIGMA, ds_rate):
    t_raw = normalize_traces(traces_raw)
    t_raw_offset = normalize_traces(traces_raw_offset)
        
    r_in = gaussian_filter1d(t_raw, axis=1, sigma=SIGMA)
    r_tar = gaussian_filter1d(t_raw_offset, axis=1, sigma=SIGMA)

    u_in = gaussian_filter1d(u_in_raw, sigma=SIGMA)

    r_in = block_reduce(r_in, block_size=(1, ds_rate), func=np.mean, cval=np.mean(r_in))
    r_tar = block_reduce(r_tar, block_size=(1, ds_rate), func=np.mean, cval=np.mean(r_tar))
    u_in = block_reduce(u_in, block_size=(1, ds_rate), func=np.mean, cval=np.mean(u_in))
